In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

In [2]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('./dataset/'))

In [3]:
# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [4]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg n
labels = [p.split(os.path.sep)[-2] for p in image_path]

In [5]:
# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

In [6]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected lay
model = VGG16(weights='imagenet', include_top=False)

In [7]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    list_image.append(image)
list_image = np.vstack(list_image)

In [8]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected lay
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [9]:
# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

In [10]:
# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

In [11]:
# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

In [12]:
# Output layer với softmax activation
fcHead = Dense(17, activation='softmax')(fcHead)

In [13]:
# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = model = Model(inputs=baseModel.input, outputs=fcHead)

In [14]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, \
test_size=0.2, random_state=42)

In [15]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, \
width_shift_range=0.1, height_shift_range=0.1, \
shear_range=0.2, zoom_range=0.2, horizontal_flip=True, \
fill_mode='nearest')

In [16]:
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [19]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train)//32,validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),validation_steps=len(X_test)//32,epochs=numOfEpoch)

Epoch 1/25


ValueError: in user code:

    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /home/nguyenhuu/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/compile_utils.py

In [ ]:
#unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True
numOfEpoch = 35
opt = SGD(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32),
steps_per_epoch=len(X_train)//32,
validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
validation_steps=len(X_test)//32,
epochs=numOfEpoch)